In [1]:
# hide
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [2]:
# default_exp staking

<IPython.core.display.Javascript object>

# Staking
> Automated staking functionality for Numerai Classic and Numerai Signals.

In [3]:
# hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

In [4]:
#export
import numpy as np
from typing import Union
from tqdm.auto import tqdm
from numerapi import NumerAPI
from typeguard import typechecked
from rich import print as rich_print

from numerblox.key import Key, load_key_from_json

<IPython.core.display.Javascript object>

## 0. BaseStaker

Staking functionality that is the same for both Numerai Classic and Numerai Signals.

Note that using staking functionality requires a `Key` object as input argument.

In [5]:
#export
@typechecked
class BaseStaker:
    """
    Base class for staking functionality which holds in both Numerai Classic and Signals.

    :param key: a numerai-blocks Key object \n
    :param tournament_number: ID of the tournament (8 for Numerai Classic and 11 for Numerai Signals).
    """
    def __init__(self, key: Key, tournament_number: int, *args, **kwargs):
        self.tournament_number = tournament_number
        self.api = NumerAPI(public_id=key.pub_id, secret_key=key.secret_key, *args, **kwargs)

    def stake_change(self, model_name: str, amount: Union[float, str]):
        """
        Change arbitrary stake amount.
        :param model_name: Lowercase raw model name (For example, 'integration_test').
        :param amount: NMR amount to increase or decrease.
        negative number = Stake decrease.
        positive number = Stake increase.
        """
        action = 'decrease' if amount < 0 else 'increase'
        color = 'red' if amount < 0 else 'green'
        model_id = self._get_model_id(model_name=model_name)
        rich_print(f"[bold {color}]{action[:-1]}ing[/bold {color}] stake by amount '[bold {color}]{amount}[/bold {color}]' for model: [bold blue]'{model_name}'[/bold blue] (id='{model_id}')!")
        self.api.stake_change(nmr=amount, action=action,
                              model_id=model_id, tournament=self.tournament_number)

    def stake_drain_all(self):
        """
        -WARNING- This function will remove all stakes for all models in your account!!! -WARNING-
        User must answer prompt with 'Y' to confirm stake drain.
        """
        prompt = input(f"WARNING: You are about to remove all stakes for all your models! Are you sure? [Y/n]")
        if prompt == "Y":
            model_names = list(self.get_model_mapping.keys())
            for name in tqdm(model_names, desc="Full stake drain!"):
                self.stake_drain_single(model_name=name)
        else:
            rich_print("Aborting full stake draining.")

    def stake_drain_single(self, model_name: str):
        """
        -WARNING- This function removes your full stake for a given model name!!! -WARNING-
        :param model_name: Lowercase raw model name (For example, 'integration_test').
        """
        model_id = self._get_model_id(model_name=model_name)
        rich_print(f":warning: Draining stake for model: '{model_name}' (id: '{model_id}' :warning:")
        self.api.stake_drain(model_id=model_id, tournament=self.tournament_number)

    def _get_model_id(self, model_name: str) -> str:
        """
        Get ID needed for staking.
        :param model_name: Lowercase raw model name (For example, 'integration_test').
        """
        return self.get_model_mapping[model_name]

    @property
    def get_model_mapping(self) -> dict:
        """ Mapping between raw model names and model IDs. """
        return self.api.get_models()

    @property
    def available_nmr(self):
        """ Get amount of NMR that is available in your local wallet. """
        return np.float64(self.api.get_account()['availableNmr'])

<IPython.core.display.Javascript object>

#### Example usage: basic functionality

In [6]:
key = load_key_from_json("test_assets/test_credentials.json")
# Numerai Classic tournament number = 8
base_staker = BaseStaker(key=key, tournament_number=8)

🔑 Numerai Auth key initialized with pub_id = 'UFVCTElDX0lE' 🔑

<IPython.core.display.Javascript object>

Mapping between raw model names and model IDs.

In [7]:
# mapping
# base_staker.get_model_mapping

<IPython.core.display.Javascript object>

Check NMR in local wallet.

In [8]:
# nmr
# base_staker.available_nmr

<IPython.core.display.Javascript object>

Change stake for model "test".

In [9]:
# base_staker.stake_change("test", -0.01)
# base_staker.stake_change("test", 0.01)

<IPython.core.display.Javascript object>

WARNING! This method will remove the full stake for a given model name!

In [10]:
# drain_single
# base_staker.stake_drain_single(model_name="test")

<IPython.core.display.Javascript object>

WARNING! This method will remove all stakes for all your models!

In [11]:
# drain_all
# base_staker.stake_drain_all()

<IPython.core.display.Javascript object>

## 1. Numerai Classic

The option to get a mapping from model names to NMR staked is only supported for Numerai Classic in `numerapi`. The reason for this is that unfortunately, `numerapi` does not support `NumerAPI().stake_get` for Numerai Signals models.
Future versions will support this feature for Numerai Signals.

In [12]:
#export
class NumeraiClassicStaker(BaseStaker):
    """ Staking functionality specific to Numerai Classic (tournament number 8). """
    CLASSIC_TOURNAMENT_NUMBER = 8
    def __init__(self, key: Key, *args, **kwargs):
        super().__init__(key=key, tournament_number=self.CLASSIC_TOURNAMENT_NUMBER,
                         *args, **kwargs
                         )

    @property
    def get_stakes(self) -> dict:
        """
        Get mapping of stakes for all models.
        Example output:
            {
            "my_model_1": 20.2,
            "my_model_2": 100.5,
            "my_model_3: 0.01
            }
        """
        stakes = [self._get_single_stake(model_name=model_name) for model_name in self.get_model_mapping.keys()]
        return {name: stake for name, stake in zip(self.get_model_mapping, stakes)}

    def _get_single_stake(self, model_name: str) -> np.float64:
        return np.float64(self.api.stake_get(username=model_name))

<IPython.core.display.Javascript object>

#### Example usage: Numerai Classic functionality

In [13]:
classic_staker = NumeraiClassicStaker(key=key)
# Numerai Classic tournament number = 8
assert classic_staker.CLASSIC_TOURNAMENT_NUMBER == 8

<IPython.core.display.Javascript object>

Get all your models with corresponding NMR staked (dict).

In [14]:
# stakes
# classic_staker.get_stakes

<IPython.core.display.Javascript object>

## 2. Numerai Signals

In [15]:
#export
class NumeraiSignalsStaker(BaseStaker):
    """ Staking functionality specific to Numerai Signals (tournament number 11). """
    SIGNALS_TOURNAMENT_NUMBER = 11
    def __init__(self, key: Key, *args, **kwargs):
        super().__init__(key=key,
                         tournament_number=self.SIGNALS_TOURNAMENT_NUMBER,
                         *args, **kwargs
                         )

<IPython.core.display.Javascript object>

#### Example usage: Numerai Signals functionality

Currently there is no Signals specific staking functionality.

`NumeraiSignalsStaker` just makes sure we use the correct tournament number.

In [16]:
signals_staker = NumeraiSignalsStaker(key=key)
# Numerai Classic tournament number = 11
assert signals_staker.SIGNALS_TOURNAMENT_NUMBER == 11

<IPython.core.display.Javascript object>

------------------------------------

In [17]:
# hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script

notebook2script()

Converted 00_misc.ipynb.
Converted 01_download.ipynb.
Converted 02_numerframe.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_model.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_modelpipeline.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted 10_staking.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>